In [ ]:
from _util import *
import _IS as IS
import _RL.FQI as FQI
import _RL.my_gym as my_gym
import _Ohio_Simulator as Ohio
import _analyze as analyze
import _RL.DQN as DQN
import _cartpole as cartpole
from coinDice import run_neural_coin_dice as run_coin

os.environ["OMP_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["OPENBLAS_NUM_THREADS"] = "1"
os.environ["VECLIB_MAXIMUM_THREADS"] = "1"
os.environ["NUMEXPR_NUM_THREADS"] = "1"
from arch.bootstrap import IIDBootstrap

reload(run_coin)
reload(my_gym)
reload(IS)

1. we can either store the data generated from TR, or re-generate it 

In [ ]:
################################################################################################
############################## generate data ##############################################
################################################################################################
unique_setting = "cart_IS"
tau = 0.3
for N in [10, 15, 20]:
    ###################################################################################################################
    n_gpu = 8
    rep = 200

    ########################################################################
    T = 300
    gamma = .98
    ################################################################################################################################################ 
    ###################################################### Default Settings ######################################################
    ################################################################################################################################################
    setting = EST()[7:9] + EST()[10:13]+ "_"  + unique_setting + "_" + "N" + str(N) + "_" + "T" + str(T) + "_" +  "tau" + "_" + str(tau)
    printR("setting: {}".format(setting))
    ################################################################################################################################################

    data_path = "data/" + "1103_cart_T300_Q_N" + str(N) + "_T300_initQ0.99_tau_" + str(tau) + "Qs"
    ##################################################################################################################
    weight_path = "target_policies/" + "cartpole" + "g" + str(0.99)
    tp_path = weight_path + "/iter" + str(20000)

    pi1 = DQN.DQN_gym(num_states = 4, num_actions = 2
                      , hidden_units = [256, 256], gamma = gamma
                      , gpu_number = 0)
    pi1.model.load_weights(tp_path)
    #########################################################
    pi_behav = my_gym.softmax_policy(pi1, tau)

    gym_eval = my_gym.GymEval(random_pi = True)

    try:
        cart_data = load(data_path)
        trajs_train_resp = cart_data["trajs_train_resp"]
        V_true = cart_data["V_true"]
    except:
        printR("error")
    ################################################################################################
    ############################## calculate step-wise IS ##############################################
    ################################################################################################
    V_seeds = IS.cal_step_IS(trajs_train_resp, gamma, pi_behav, pi1)
    ################################################################################################
    ############################## calculate CIs ##############################################
    ################################################################################################
    ######################################  
    ## Bernstein_CI: perfect but too wide
    CI_seeds = []
    cnt_09, cnt_095, width09, width095 = [], [], [], []
    for V_trajs, V_min, V_max in V_seeds:
        CIs_this = IS.cal_Bernstein_CI(Ys = V_trajs, V_min = V_min, V_max = V_max)
        CI_seeds.append(CIs_this) # [90%, 95%]
        CI09, CI095 = CIs_this
        cnt_09.append((CI09[0] < V_true and CI09[1] > V_true))
        width09.append(CI09[1] - CI09[0])
        cnt_095.append((CI095[0] < V_true and CI095[1] > V_true))
        width095.append(CI095[1] - CI095[0])
    res_IS_Bernstein = {"all_CI" : CI_seeds, "width" : [mean(width09), mean(width095)], "freq" : [mean(cnt_09), mean(cnt_095)]}

    dump(res_IS_Bernstein, "res/" + unique_setting + "_Bernstein" + "_N" + str(N) + "_tau" + str(tau))
    ######################################
    ## Bootstrap CIs
    CI_seeds = []
    cnt095, cnt09, width09, width095 = [], [], [], []
    for seed in range(200):
        V_trajs, _, _ = V_seeds[seed]
        bs = IIDBootstrap(arr(V_trajs))
        ci095 = bs.conf_int(lambda x: x.mean(), 1000, method='bca', size = 0.95)
        ci09 = bs.conf_int(lambda x: x.mean(), 1000, method='bca', size = 0.9)
        CI_seeds.append([ci09, ci095])
        cnt095.append((ci095[0][0] < V_true and ci095[1][0] > V_true))
        cnt09.append((ci09[0][0] < V_true and ci09[1][0] > V_true))
        width09.append(ci09[1] - ci09[0])
        width095.append(ci095[1] - ci095[0])
    res_IS_Bootstrap = {"all_CI" : CI_seeds, "width" : [mean(width09), mean(width095)], "freq" : [mean(cnt09), mean(cnt095)]}

    ################################################################################################
    ############################## save results ##############################################
    ################################################################################################
    dump(res_IS_Bootstrap, "res/" + unique_setting + "_Bootstrap" + "_N" + str(N) + "_tau" + str(tau))